In [1]:
import nltk
import pickle
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [15]:
pTrees = nltk.corpus.treebank.parsed_sents()
%time len(pTrees) #hm... so counting through all of the trees takes about 5 seconds, but this isn't even touching any part of the tree.
pTrees[0].pretty_print()

Wall time: 2.57 s


3914

                                                     S                                                                         
                         ____________________________|_______________________________________________________________________   
                        |                                               VP                                                   | 
                        |                        _______________________|___                                                 |  
                      NP-SBJ                    |                           VP                                               | 
         _______________|___________________    |     ______________________|______________________________________          |  
        |          |              ADJP      |   |    |        |                PP-CLR                              |         | 
        |          |           ____|____    |   |    |        |          ________|_________          

In [3]:
pTreeProductions = []
for tree in pTrees:
    pTreeProductions.append(tree.productions())

In [17]:
len(pTreeProductions)
pTreeProductions[0]

3914

[S -> NP-SBJ VP .,
 NP-SBJ -> NP , ADJP ,,
 NP -> NNP NNP,
 NNP -> 'Pierre',
 NNP -> 'Vinken',
 , -> ',',
 ADJP -> NP JJ,
 NP -> CD NNS,
 CD -> '61',
 NNS -> 'years',
 JJ -> 'old',
 , -> ',',
 VP -> MD VP,
 MD -> 'will',
 VP -> VB NP PP-CLR NP-TMP,
 VB -> 'join',
 NP -> DT NN,
 DT -> 'the',
 NN -> 'board',
 PP-CLR -> IN NP,
 IN -> 'as',
 NP -> DT JJ NN,
 DT -> 'a',
 JJ -> 'nonexecutive',
 NN -> 'director',
 NP-TMP -> NNP CD,
 NNP -> 'Nov.',
 CD -> '29',
 . -> '.']

In [5]:
def getAllInstances(wordType):
    instances = []
    for tree in pTreeProductions:
        for rule in tree:
            text = str(rule)
            left, right = text.split(" -> ")
            #print("Left, Rule:")
            #print(left)
            #print(rule)
            #print()
            if(left == wordType):
                instances.append(rule)
    return instances

In [6]:
adjs = getAllInstances('JJ')
len(adjs)

5834

In [7]:
file = open('cats.pkl','rb') 
cats = pickle.load(file)

In [8]:
for cat in cats:
    if(cat == 'NN'):
        print(cat)

NN


In [9]:
cats = list(cats)
type(cats)

list

In [10]:
cat_lists = [] #list of lists
for cat in sorted(cats):
    print("Loading all instances of: ", cat)
    %time cat_lists.append(getAllInstances(cat))

Loading all instances of:  ,
Wall time: 658 ms
Loading all instances of:  .
Wall time: 598 ms
Loading all instances of:  ADJP
Wall time: 537 ms
Loading all instances of:  ADVP
Wall time: 554 ms
Loading all instances of:  CC
Wall time: 551 ms
Loading all instances of:  CD
Wall time: 696 ms
Loading all instances of:  DT
Wall time: 572 ms
Loading all instances of:  EX
Wall time: 561 ms
Loading all instances of:  FRAG
Wall time: 543 ms
Loading all instances of:  IN
Wall time: 654 ms
Loading all instances of:  JJ
Wall time: 619 ms
Loading all instances of:  JJR
Wall time: 536 ms
Loading all instances of:  JJS
Wall time: 566 ms
Loading all instances of:  MD
Wall time: 532 ms
Loading all instances of:  NN
Wall time: 642 ms
Loading all instances of:  NNP
Wall time: 528 ms
Loading all instances of:  NNPS
Wall time: 523 ms
Loading all instances of:  NNS
Wall time: 544 ms
Loading all instances of:  NP
Wall time: 519 ms
Loading all instances of:  NP-TMP
Wall time: 523 ms
Loading all instances of: 

In [13]:
cat_lists[5][0]

CD -> '61'

In [127]:
def getWordProb(wordType, key, isLeaf):
    tagIndex = 0
    for tag in cat_lists:
        try: #needs this try/except because there's one tag that's just empty, the line below errors out.
            raw = str(cat_lists[tagIndex][0])
        except:
            continue
        if(wordType == raw.split(" -> ")[0]): #now at this point, we're on the list of entries for the tag. count hits/total
            hits = 0 #^ this must be true for 'hits' to initialize, if it isn't then the tag is not in the sample
            total = 0
            for rule in cat_lists[tagIndex]:
                #print(rule)
                text = str(rule)
                left, right = text.split(" -> ")
                #print(left, right)
                if(isLeaf):
                    if(key == right[1:-1] or key == right[1:-1].lower()):
                        hits += 1
                else:
                        if(key == right):
                            hits += 1
                total += 1
        tagIndex += 1
    try:
        print("Hits: ", hits, "\nTotal: ", total)
    except:
        print("This tag was not found in the sample of the Penn Treebank.") #FW, LS, POS, UH, RP, SYM
        return
    return hits/total

In [192]:
getWordProb('NN', 'money', True)

Hits:  56 
Total:  13166


0.00425337991797053